In [1]:
from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
import numpy as np, pandas as pd

In [2]:
#can only go to 10000 items because after that the URI ha no discernable distinction
pages = np.arange(1, 9901, 100)

In [31]:
#initialize empty lists to store the variables scraped
names = []
actor_or_actress_list = []
about_list = []
image_list= []
name_list = []

In [37]:
# Crawling the details of celebs under <div class="lister-item-content"> element.
# Deeply inspect the elements , you can easily inspect the details out of it.
for page in pages:
    
    #get request
    response = get("https://www.imdb.com/list/ls002913270/" + "?sort=list_order,asc&mode=detail&" + "page=" + str(page))
    
    sleep(randint(8,15))
     
    #throw warning for status codes that are not 200
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    #parse the content of current iteration of request
    page_html = BeautifulSoup(response.text, 'html.parser')
        
    actor_containers = page_html.find_all('div', class_ = 'lister-item-content')
    
    #extract the 100 movies for that page
    for container in actor_containers:
        
        # name
        name = container.h3.a.text
        names.append(name)

        #about
        about = container.p.text
        about_list.append(about)

In [38]:
print("\n Size of Name List = " , len(names))
print("\n Size of About List = ", len(about_list))


 Size of Name List =  100

 Size of About List =  100


In [47]:
# cleaning the data from names creawled from IMDb link
temp_name_list = []
for item in names:
    temp_name_list.append(item.replace('\n' , ''))

print(temp_name_list)

[' Shah Rukh Khan', ' Aamir Khan', ' Salman Khan', ' Katrina Kaif', ' Kareena Kapoor', ' Karisma Kapoor', ' Hrithik Roshan', ' Priyanka Chopra', ' Amitabh Bachchan', ' Ranbir Kapoor', ' Anushka Sharma', ' Imran Khan', ' Deepika Padukone', ' Ali Zafar', ' Akshay Kumar', ' Aishwarya Rai Bachchan', ' Rani Mukherjee', ' John Abraham', ' Bipasha Basu', ' Rajinikanth', ' Asin Thottumkal', ' Dilip Kumar', ' Hema Malini', ' Madhuri Dixit', ' Govinda', ' Mahie Gill', ' Preity Zinta', ' Arshad Warsi', ' Shilpa Shetty Kundra', ' Kajol', ' Sunny Deol', ' Urmila Matondkar', ' Saif Ali Khan', ' Juhi Chawla', ' Tusshar Kapoor', ' Lara Dutta', ' Anil Kapoor', ' Jacqueline Fernandez', ' Ranveer Singh', ' Madhavan', ' Minissha Lamba', ' Shreyas Talpade', ' Shahid Kapoor', ' Sonam Kapoor', " Genelia D'Souza", ' Irrfan Khan', ' Sonakshi Sinha', ' Parineeti Chopra', ' Vidya Balan', ' Bobby Deol', ' Dharmendra', ' Zareen Khan', ' Shazahn Padamsee', ' Riteish Deshmukh', ' Neil Nitin Mukesh', ' Naseeruddin Sh

In [65]:
# cleaning the about_list removing new line character.
temp_list = []
temp_about_list= []
for item in about_list:
    temp_list.append(item.replace('\n' , ''))
    
print("\n\n\n" , temp_list)

recent_movie_name = []
actor_or_actress_temp = []

def cleanup(temp_list):
    recent_movie = []
    actor_or_actress_temp = []
    for i in range(0,len(temp_list)):
        # for removing whitespaces
        removed_ws_string = temp_list[i].replace(" " , "")
        # splitting a perticular string w.r.t '|'
        actor_or_actress = removed_ws_string.split('|')
        # assign the splitted word to each or the list
        actor_or_actress_temp.append(actor_or_actress[0])
        recent_movie.append(actor_or_actress[1])
    
    return actor_or_actress_temp , recent_movie

actor_or_actress_temp , recent_movie_name = cleanup(temp_list)
    
print("\n\n\n" , recent_movie_name ,"\n\n\n" ,  actor_or_actress_temp)




 ['                        Actor | Don 2 ', '                        Actor | Taare Zameen Par ', '                        Actor | Sultan ', '                        Actress | Zindagi Na Milegi Dobara ', '                        Actress | Jab We Met ', '                        Actress | Dil To Pagal Hai ', '                        Actor | Super 30 ', '                        Actress | Baywatch ', '                        Actor | Black ', '                        Actor | Rockstar ', '                        Actress | Jab Tak Hai Jaan ', '                        Actor | Jaane Tu... Ya Jaane Na ', '                        Actress | Yeh Jawaani Hai Deewani ', '                        Actor | Teefa in Trouble ', '                        Actor | Hera Pheri ', '                        Actress | Jodhaa Akbar ', '                        Actress | Tumhare Liye ', '                        Actor | Dhoom ', '                        Actress | Race ', '                        Actor | Enthiran ', ' 

In [66]:
celeb_details = pd.DataFrame( data = {"Celeb Name" : temp_name_list , "Actor/Actress" : actor_or_actress_temp , "Recent Movie" : recent_movie_name })
celeb_details.to_csv('celeb_details.csv' , sep=',' , index=False)

In [32]:
# Crawling the link of images for clebs under <div class="lister-item-image"> element
for page in pages:
    
    #get request
    response = get("https://www.imdb.com/list/ls002913270/" + "?sort=list_order,asc&mode=detail&" + "page=" + str(page))
    
    sleep(randint(8,15))
     
    #throw warning for status codes that are not 200
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    #parse the content of current iteration of request
    page_html = BeautifulSoup(response.text, 'html.parser')
        
    image_containers = page_html.find_all('div', class_ = 'lister-item-image')
    
    #extract the 100 images for that page
    for container in image_containers:
        
        #image_link
        image_link = container.find('img' , attrs = {'height' : '209' })['src']
        image_list.append(image_link)
        
        #name_list
        a_name = container.find('img' , attrs = {'height' : '209' })['alt']
        name_list.append(a_name)

In [33]:
print(len(image_list))
print(len(name_list))
for i in range(0, len(image_list)):
    print(name_list[i] , "\n" , image_list[i])

100
100
Shah Rukh Khan 
 https://m.media-amazon.com/images/M/MV5BZDk1ZmU0NGYtMzQ2Yi00N2NjLTkyNWEtZWE2NTU4NTJiZGUzXkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_UY209_CR3,0,140,209_AL_.jpg
Aamir Khan 
 https://m.media-amazon.com/images/M/MV5BMjAwMjk3NDUzN15BMl5BanBnXkFtZTcwNjI4MTY0NA@@._V1_UX140_CR0,0,140,209_AL_.jpg
Salman Khan 
 https://m.media-amazon.com/images/M/MV5BMjMwNTIxODg0OF5BMl5BanBnXkFtZTgwODg2NzM0OTE@._V1_UY209_CR7,0,140,209_AL_.jpg
Katrina Kaif 
 https://m.media-amazon.com/images/M/MV5BYjFjZGMwNWEtYWYwZi00NjdiLTk3NDMtOGVlOTkwOWFhYjE0XkEyXkFqcGdeQXVyMTA0NTY2NTIz._V1_UY209_CR34,0,140,209_AL_.jpg
Kareena Kapoor 
 https://m.media-amazon.com/images/M/MV5BMjAyNjM4NTAwN15BMl5BanBnXkFtZTcwNDkxNzQzNg@@._V1_UY209_CR87,0,140,209_AL_.jpg
Karisma Kapoor 
 https://m.media-amazon.com/images/M/MV5BMzY4NTkxMzc3OF5BMl5BanBnXkFtZTgwMTc1MzY3OTE@._V1_UY209_CR2,0,140,209_AL_.jpg
Hrithik Roshan 
 https://m.media-amazon.com/images/M/MV5BZjYwYjE4YjQtMTFmMi00NTlhLThjZjUtZTIyMTczMGU2ZDE5XkEyXkFqcGdeQXVyNzM2NzM0ND

In [34]:
actor_details = pd.DataFrame( data = {"Actor Name" : name_list , "Actor Image Link" : image_list})
actor_details.to_csv('actor_image_link.csv' , sep=',' , index=False)

In [ ]:
# Now, we have the list of image links for celebs we scrapped.
# So we can easily download images with the help of wget and save in a folder.
import wget

for i in range(0 , len(image_link)):
    url = image_link[i]
    wget.download(url, 'C:/Users/KP/Documents/Programs/actors/'+name_list[i])